## Scikit Learn Pipelines and Preprocessing steps
Since we now know about different scikit learn functions, let's try to learn how do we tie different steps together! 

By the end of this tutorial you'll know:
1. Different pre-processing steps which can be done on our data
2. How to tie the transformers and estimators together into a pipeline
3. How to make your own transformer
4. Finally, how to tie it all up together!

### Scikit Learn Pipelines
As per the documentation, 
"Sequentially apply a list of transforms and a final estimator.
    Intermediate steps of the pipeline must be 'transforms', that is, they
    must implement fit and transform methods.
    The final estimator only needs to implement fit."
For more information click here : (https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/pipeline.py)

Let's start by loading our data: